In [1]:
import sys
import os
sys.path.append("../")

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]= ""

In [3]:
import pandas as pd
import numpy as np
example =pd.read_parquet('../../dataset/train_landmarks/1019715464.parquet')
classes,counts = np.unique([x.split("_")[1] for x in example.columns[1:]],return_counts=True)

def sort_func(x):
    p = x.split("_")[-1]
    return "_".join(x.split("_")[1:-1])+ "{:04d}".format(int(p))

COLUMNS_SEQUENCE=sorted(example.columns[1:],key=sort_func)

In [4]:
# sys.path.append("../../runs/ctc_kaggle_copy/fold3")

In [5]:
from signet.configs.ctc_loss_with_downsampled_deploy_strongeraugs import ctc_loss_encdec_params
# from signet.models.feature_extractor import Cnn1dMhsaFeatureExtractor
from signet.models.feature_extractor_downsampled import Cnn1dMhsaFeatureExtractor

from signet.models.encoder_decoder import EncoderDecoder

from signet.dataset.preprocess import Preprocess
import tensorflow as tf

2023-07-28 14:43:21.319411: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-28 14:43:21.385782: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [20]:
CFG = ctc_loss_encdec_params()
CFG.kernel_size_downsampling = CFG.kernel_size
CFG.downsampling_strides = 2
CFG.attention_span = 0
CFG.do_downsample = True
model = Cnn1dMhsaFeatureExtractor(CFG)

In [21]:
model.load_weights("../../runs/ctc_with_masking/fold3_poseinfo_moreaugs/best_ckpt.h5")

In [22]:
class TFLiteModel(tf.Module):
    """
    TensorFlow Lite model that takes input tensors and applies:
        – a preprocessing model
        – the ISLR model 
    """

    def __init__(self, model):
        """
        Initializes the TFLiteModel with the specified preprocessing model and ISLR model.
        """
        super(TFLiteModel, self).__init__()

        # Load the feature generation and main models
        self.prep_inputs = Preprocess(CFG)
        self.islr_models   = model
    
    @tf.function(input_signature=[tf.TensorSpec(shape=[None,len(COLUMNS_SEQUENCE)], dtype=tf.float32, name='inputs')])
    def __call__(self, inputs):
        x = tf.reshape(inputs,(-1,543,3))
        mask = tf.math.logical_not(tf.reduce_all(tf.math.is_nan(tf.gather(x,CFG.POINT_LANDMARKS,axis=1)), axis=[-2,-1]))
        mask = tf.reshape(mask, [-1])  
        x = tf.boolean_mask(x, mask, axis=0)
        x = self.prep_inputs(tf.cast(x, dtype=tf.float32))[0]
        seqlen = tf.shape(x)[0]
        if seqlen < CFG.max_len:
            padding = tf.tile([x[-1, :]], [CFG.max_len - seqlen, 1])
            x = tf.concat([x, padding], axis=0)
        elif seqlen > CFG.max_len:
            x = x[:CFG.max_len, :]
        outputs = model(tf.expand_dims(x,0))[0]
        indices = tf.math.argmax(outputs, axis=-1)  # [num_seq,]
        tensor = tf.reshape(indices, [-1])  # Flatten tensor to 1D
        diff = tf.concat([tf.constant([1], dtype=tensor.dtype), tensor[1:] - tensor[:-1]], axis=0)
        unique_indices = tf.where(diff != 0)[:, 0]
        unique_values = tf.gather(tensor, unique_indices)
        outputs = tf.gather(unique_values,tf.where(unique_values!=CFG.blank_index))
        outputs = tf.reshape(outputs, [-1])
        return {'outputs': tf.one_hot(outputs,len(CFG.idx_to_char))}

In [23]:
# class TFLiteModelEncDec(tf.Module):
#     """
#     TensorFlow Lite model that takes input tensors and applies:
#         – a preprocessing model
#         – the ISLR model 
#     """

#     def __init__(self, model):
#         """
#         Initializes the TFLiteModel with the specified preprocessing model and ISLR model.
#         """
#         super(TFLiteModelEncDec, self).__init__()

#         # Load the feature generation and main models
#         self.prep_inputs = Preprocess(CFG)
#         self.islr_models   = model
    
#     @tf.function(input_signature=[tf.TensorSpec(shape=[None,len(COLUMNS_SEQUENCE)], dtype=tf.float32, name='inputs')])
#     def __call__(self, inputs):
#         """
#         Applies the feature generation model and main model to the input tensors.

#         Args:
#             inputs: Input tensor with shape [batch_size, 543, 3].

#         Returns:
#             A dictionary with a single key 'outputs' and corresponding output tensor.
#         """
#         x = self.prep_inputs(tf.cast(tf.reshape(inputs,(-1,543,3)), dtype=tf.float32))[:CFG.max_len]
#         outputs = model(x)[0]
        
#         indices = tf.math.argmax(outputs, axis=-1)
#         endidx = tf.where(tf.equal(indices, CFG.end_index))

#         if tf.size(endidx) > 0:
#             first_twenty_index = endidx[0][0]
#             indices = indices[:first_twenty_index]
        
#         return {'outputs': tf.one_hot(indices,len(CFG.idx_to_char))}

In [24]:
# x = tf.reshape(demo_raw_data,(-1,543,3))
# mask = tf.math.logical_not(tf.reduce_all(tf.math.is_nan(tf.gather(x,CFG.POINT_LANDMARKS,axis=1)), axis=[-2,-1]))
# mask = tf.reshape(mask, [-1])  
# x = tf.boolean_mask(x, mask, axis=0)
# x = Preprocess(CFG)(tf.cast(x, dtype=tf.float32))[0]
# seqlen = tf.shape(x)[0]
# if seqlen < CFG.max_len:
#     padding = tf.tile([x[-1, :]], [CFG.max_len - seqlen, 1])
#     x = tf.concat([x, padding], axis=0)
# elif seqlen > CFG.max_len:
#     x = x[:CFG.max_len, :]
# outputs = model(tf.expand_dims(x,0))

In [25]:
path = f'../../dataset/train_landmarks/5414471.parquet'

df_s = pd.read_parquet(path, engine = 'auto', columns = COLUMNS_SEQUENCE)

demo_raw_data = df_s.loc[1816796431, :][COLUMNS_SEQUENCE].values

print(f'demo_raw_data shape: {demo_raw_data.shape}, dtype: {demo_raw_data.dtype}')

demo_raw_data shape: (123, 1629), dtype: float32


In [26]:
tf_lite_model = TFLiteModel(model)

keras_model_converter = tf.lite.TFLiteConverter.from_keras_model(tf_lite_model)
tflite_model = keras_model_converter.convert()
with open('./data/model.tflite', 'wb') as f:
    f.write(tflite_model)

2023-07-28 14:44:56.193093: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,1629]
	 [[{{node inputs}}]]
2023-07-28 14:44:59.770850: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'input_6' with dtype float and shape [?,384,540]
	 [[{{node input_6}}]]
2023-07-28 14:45:06.522207: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'input_6' with dtype float and shape [?,384,540]
	 [[{{node input_6}}]]
202

INFO:tensorflow:Assets written to: /tmp/tmpd8cz5ne7/assets


INFO:tensorflow:Assets written to: /tmp/tmpd8cz5ne7/assets
2023-07-28 14:46:20.473297: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'serving_default_inputs' with dtype float and shape [?,1629]
	 [[{{node serving_default_inputs}}]]
2023-07-28 14:46:37.919735: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-07-28 14:46:37.919789: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-07-28 14:46:37.920805: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpd8cz5ne7
2023-07-28 14:46:37.976847: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-07-28 14:46:37.976873: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info 

In [27]:
import json
with open('./data/inference_args.json', 'w') as f:
     json.dump({ 'selected_columns': COLUMNS_SEQUENCE}, f)

In [29]:
"".join([CFG.idx_to_char[x] for x in tf_lite_model(demo_raw_data)["outputs"].numpy().argmax(-1)])

'3 creek house'

In [30]:
tf_lite_model = TFLiteModel(model)

keras_model_converter = tf.lite.TFLiteConverter.from_keras_model(tf_lite_model)

keras_model_converter.optimizations = [tf.lite.Optimize.DEFAULT]
keras_model_converter.target_spec.supported_types = [tf.float16]

tflite_model = keras_model_converter.convert()
with open('./data_fp16/model.tflite', 'wb') as f:
    f.write(tflite_model)


2023-07-26 08:34:23.220474: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,1629]
	 [[{{node inputs}}]]
2023-07-26 08:34:28.055839: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'input_1' with dtype float and shape [?,384,540]
	 [[{{node input_1}}]]
2023-07-26 08:34:34.101705: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'input_1' with dtype float and shape [?,384,540]
	 [[{{node input_1}}]]
202

INFO:tensorflow:Assets written to: /tmp/tmp62r2jfi4/assets


INFO:tensorflow:Assets written to: /tmp/tmp62r2jfi4/assets
2023-07-26 08:35:24.706490: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'serving_default_inputs' with dtype float and shape [?,1629]
	 [[{{node serving_default_inputs}}]]
2023-07-26 08:35:42.240173: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-07-26 08:35:42.240230: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-07-26 08:35:42.240628: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp62r2jfi4
2023-07-26 08:35:42.326869: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-07-26 08:35:42.326915: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info 

In [24]:
!cd data && zip submission.zip model.tflite inference_args.json

updating: model.tflite (deflated 8%)
updating: inference_args.json (deflated 83%)


In [38]:
!du -sh ./data/submission.zip

34M	./data/submission.zip


## Evaluate converted model

In [1]:
import sys
sys.path.append("../")
from signet.losses.metrics import normalized_levenshtein_distance,total_levenshtein_distance
import tensorflow as tf

2023-07-28 14:48:19.986008: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-28 14:48:20.049333: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
interpreter = tf.lite.Interpreter(model_path="./data/model.tflite")

In [3]:
import Levenshtein as lev
import pandas as pd
import numpy as np
import os
import sys
from tqdm import tqdm
sys.path.append("../")

In [4]:
ROOT = "../../dataset/npy_data/"
VALIDFILE = "../../dataset/folds/fold3_valid.csv"

In [5]:
df = pd.read_csv(VALIDFILE)

In [6]:
from signet.configs.base import Base 

In [7]:
CFG = Base()

In [8]:
REQUIRED_SIGNATURE = "serving_default"
REQUIRED_OUTPUT = "outputs"

found_signatures = list(interpreter.get_signature_list().keys())

prediction_fn = interpreter.get_signature_runner("serving_default")

In [9]:
df = df.sample(200)

In [10]:
import numpy as np

def remove_static_frames(frames, joints, d):
    # Calculate the absolute differences between consecutive frames
    diff_frames = np.abs(np.diff(frames, axis=0))
    
    # Select the joints of interest
    diff_frames = diff_frames[:, joints, :2]
    
    # Calculate the distance change for each joint
    joint_dist_changes = np.linalg.norm(diff_frames, axis=-1)
    
    # Check if any joint has changed position by more than the threshold d
    frame_mask = np.any(joint_dist_changes > d, axis=-1)
    # Add the first frame (since np.diff reduced the frame length by 1)
    frame_mask = np.concatenate(([True], frame_mask))
    
    # Select frames where any joint has changed position by more than d
    frames = frames[frame_mask]
    
    return frames,frame_mask
joints = CFG.POINT_LANDMARKS

In [11]:
true = []
preds = []
for file,truth in tqdm(zip(df.files.apply(lambda x: os.path.join(ROOT,x)),df.labels),total=200):
    output = prediction_fn(inputs=np.load(file).reshape(1,-1,1629))
    text = "".join(CFG.idx_to_char[int(x)] for x in output["outputs"].argmax(1))
    true.append(truth)
    preds.append(text)

100%|██████████| 200/200 [02:17<00:00,  1.46it/s]


In [13]:
normalized_levenshtein_distance(true,preds)

0.739095744680851

In [ ]:
0.5337934263925433 for the best 0.72 modela
0.6660 for 0.55 model

In [13]:
# normlev = []
# thresholds = []
# for thres in tqdm(range(0,1)):
#     true = []
#     preds = []
#     for file,truth in zip(df.files.apply(lambda x: os.path.join(ROOT,x)),df.labels):
#         fr,mask = remove_static_frames(np.load(file),joints,thres/1000)
#         output = prediction_fn(inputs=fr.reshape(-1,1629))
#         text = "".join(CFG.idx_to_char[int(x)] for x in output["outputs"].argmax(1))
#         true.append(truth)
#         preds.append(text)
#     normlev.append(normalized_levenshtein_distance(true,preds))
#     thresholds.append(thres/1000)
#     print(normlev[-1])

In [14]:
def predict(file):
    output = prediction_fn(inputs=np.load(file).reshape(-1,1629))
    text = "".join(CFG.idx_to_char[int(x)] for x in output["outputs"].argmax(1))
    return text

In [31]:
# import swifter
# df.files.apply(lambda x: os.path.join(ROOT,x)).swifter.progress_bar(
#     enable=True
# ).apply(predict)